# Universidade Federal do Rio Grande do Norte


## Programa de Pós-Graduação em Engenharia Elétrica e de Computação
## EEC1509 - Aprendizagem de Máquina


# Group

## João Lucas Correia Barbosa de Farias

## Júlio Freire Peixoto Gomes


# Project 2 - Traffic Sign Recognition


## About the Project
This project is divided in 6 files including this one, where each one represents one step in the process of deploying a machine learning algorithm. In this case, we chose a Neural Network algorithm as Classifier. The goal is to explore learning, generalization and batch-normalization techniques and compare results.

The dataset has over 50k images of traffic signs. Our goal is to predict which sign a specific image refers to.


### The details about the dataset are shown below.

The German Traffic Sign Benchmark is a multi-class, single-image classification challenge held at the International Joint Conference on Neural Networks (IJCNN) 2011.

*   Single-image, multi-class classification problem
*   More than 40 classes
*   More than 50,000 images in total
*   Large, lifelike database

For more information, visit:

https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign

Also, for each class, that is a respective shape, color and sign id's. They are describred as follows:



1.   Shape ID
  *   0: red
  *   1: blue
  *   2: yellow
  *   3: white
2.   Color ID
  *   0: triangle
  *   1: circle
  *   2: diamond
  *   3: hexagon
  *   4: inverse-triangle
3.   Sign ID
  *   float: value according to Ukranian Traffic Rule

## The dataset was taken from Kaggle:
https://www.kaggle.com/datasets/uciml/red-wine-quality-cortez-et-al-2009

# 1.0 Install and Load Libraries


In [4]:
%%capture
# install wandb
!pip install wandb

In [12]:
import logging
import pandas as pd
import numpy as np
import h5py
from PIL import Image
import wandb
import joblib
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import fbeta_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from imblearn.metrics import geometric_mean_score
import matplotlib.pyplot as plt
from tensorflow import keras

# 2.0 Test
In this last step, we will test the exported model against the test set and analyze the results.

## 2.1 Login to Weights & Biases

In [6]:
# login to wandb
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## 2.2 Classes Used in Pipeline

It is necessary to define (again) the classes used in the pipeline creation so that joblib can work properly.

In [64]:
class Normalize():
  def __init__(self, value=255):
    self.value = value

  def fit(self, X):
    return self

  def transform(self, X):
    return X/self.value

class FeatureSelector(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X

class NumericalTransformer(BaseEstimator, TransformerMixin):
    # normalize = False: no scaler
    # normalize = True: normalize RBG values
    def __init__(self, normalize=True, image_height=30, image_width=30):
        self.normalize = normalize
        self.image_height = image_height
        self.image_width = image_width
        self.scaler = None

    def fit(self, X, y=None):
        if self.normalize:
          self.scaler = Normalize(255)
          self.scaler.fit(X)
        return self

    # transforming numerical features
    def transform(self, X, y=None):
        X_copy = []

        for img in X:
          image = Image.fromarray(img)
          image = image.resize((self.image_height,self.image_width))
          img = np.array(image)
          X_copy.append(img)

        X_copy = np.array(X_copy)

        if self.normalize:
          X_copy = self.scaler.transform(X_copy)

        return X_copy

## 2.3 Importing Model and Target Encoder

In [65]:
# name of the artifact related to test dataset
artifact_test_name = "traffic_sign_recognition/test.h5:latest"

# name of the artifact related to test dataset
artifact_test_labels_name = "traffic_sign_recognition/test_labels.csv:latest"

# name of the pipeline artifact
artifact_pipeline_name = "traffic_sign_recognition/pipeline:latest"

# name of the model artifact
artifact_model_name = "traffic_sign_recognition/model.h5:latest"

In [66]:
# configure logging
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(message)s",
                    datefmt='%d-%m-%Y %H:%M:%S')

# reference for a logging obj
logger = logging.getLogger()

In [67]:
# initiate the wandb project
run = wandb.init(project="traffic_sign_recognition",job_type="test")

In [72]:
logger.info("Downloading and reading test artifact...")

artifact_test = run.use_artifact(artifact_test_name)
artifact_test_labels = run.use_artifact(artifact_test_labels_name)

artifact_test_path = artifact_test.file()
artifact_test_labels_path = artifact_test_labels.file()

df_test_labels = pd.read_csv(artifact_test_labels_path)

image_data = []
test_labels_aux = []

with h5py.File(artifact_test_path, 'r') as hf:
  images = list(hf.keys())
  for img in images:
    df_aux = df_test_labels[df_test_labels['path'] == str(img)]
    test_labels_aux.append(df_aux['label'])
    data = hf[img]
    data_array = np.array(data)
    image_data.append(np.array(data_array))

test = np.array(image_data)
test_labels = np.array(test_labels_aux)

print(f"test.shape: {test.shape}")
print(f"test_labels.shape: {test_labels.shape}")

00004.png
11
00005.png
38
00007.png
12
00009.png
35
00015.png
9
00017.png
20
00018.png
27
00020.png
4
00026.png
13
00028.png
9
test.shape: (3722,)
test_labels.shape: (3722, 1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [77]:
test_labels = test_labels.reshape((test_labels.shape[0],))
print(test_labels.shape)
print(test_labels)

(3722,)
[11 38 12 ... 25  6  7]


In [78]:
# downloading inference artifact using joblib
logger.info("Downloading and load the exported pipeline")
pipeline_export_path = run.use_artifact(artifact_pipeline_name).file()
pipeline = joblib.load(pipeline_export_path)

In [79]:
best_model = wandb.restore('model-best.h5', run_path="traffic_sign_recognition/34xmkdgt")

In [80]:
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, Dropout, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.layers import Activation

from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy


def load_model():
    # setup wandb
    # neural network layers
    lenet5 = Sequential()

    # testing the effects of batch-normalization (before activation function)
    if 2 != 1:
        lenet5.add(Conv2D(6, (5, 5), strides=1, kernel_initializer='glorot_uniform', activation='tanh',
                          input_shape=(30, 30, 3), padding='same'))
    else:
        lenet5.add(
            Conv2D(6, (5, 5), strides=1, kernel_initializer='glorot_uniform', input_shape=(30, 30, 3), padding='same'))
        lenet5.add(BatchNormalization())
        lenet5.add(Activation('tanh'))

    # testing the effects of batch-normalization (after activation function)
    if 2 == 2:
        lenet5.add(BatchNormalization())

    lenet5.add(AveragePooling2D())
    lenet5.add(Conv2D(16, (5, 5), strides=1, activation='tanh', padding='valid'))

    # testing the addition of dropout layers
    if False:
        lenet5.add(Dropout(rate=0.5))

    lenet5.add(AveragePooling2D())  # S4
    lenet5.add(Flatten())  # Flatten
    lenet5.add(Dense(120, activation='tanh'))  # C5

    # testing the addition of dropout layers
    if False:
        lenet5.add(Dropout(rate=0.5))

    lenet5.add(Dense(84, activation='tanh'))  # F6

    # testing the addition of dropout layers
    if False:
        lenet5.add(Dropout(rate=0.25))
    lenet5.add(Dense(43, activation='softmax'))  # Output layer
    return lenet5

def charge_model(model):
    best_model = wandb.restore('model-best.h5', run_path="traffic_sign_recognition/34xmkdgt")
    model.load_weights(best_model.name)
    return model

model = load_model()
model_charged = charge_model(model)

In [89]:
# downloading inference artifact using joblib
logger.info("Downloading and load the exported model")
model_export_path = run.use_artifact(artifact_model_name).file()
model = keras.models.load_model(best_model.name)

## 2.4 Passing Test Set Through Pipeline and Model

In [81]:
x_test = pipeline.transform(test)
y_test = test_labels

In [82]:
print(x_test.shape)

(3722, 30, 30, 3)


In [90]:
# predicting the rating of the test set
logger.info("Infering...")
# predict = model.predict(x_test)
predict = model.predict(x_test)


# predict has shape (xx, 43)
# we need to find argmax of output to compare with y_test
predict_new = []

for pred in predict:
  predict_new.append(pred.argmax())

predict = np.array(predict_new)

# Evaluation Metrics
logger.info("Test Evaluation Metrics:")
acc = accuracy_score(y_test, predict)
fbeta = fbeta_score(y_test, predict, beta=1, average = 'weighted', zero_division=1)
precision = precision_score(y_test, predict, average = 'weighted', zero_division=1)
recall = recall_score(y_test, predict, average = 'weighted', zero_division=1)

logger.info("Test Accuracy: {}".format(acc))
logger.info("Test Precision: {}".format(precision))
logger.info("Test Recall: {}".format(recall))
logger.info("Test F1: {}".format(fbeta))

run.summary["Acc"] = acc
run.summary["Precision"] = precision
run.summary["Recall"] = recall
run.summary["F1"] = fbeta

In [91]:
# comparing the accuracy, precision, recall with previous ones
print(classification_report(y_test,predict))

              precision    recall  f1-score   support

           0       1.00      0.65      0.79        17
           1       0.92      0.94      0.93       232
           2       0.90      0.97      0.93       251
           3       0.89      0.92      0.90       134
           4       0.91      0.93      0.92       202
           5       0.90      0.87      0.88       168
           6       0.91      0.53      0.67        40
           7       0.88      0.87      0.88       111
           8       0.89      0.92      0.91       114
           9       0.93      0.98      0.96       133
          10       0.98      0.97      0.98       206
          11       0.88      0.93      0.90       138
          12       0.95      0.95      0.95       207
          13       0.96      1.00      0.98       207
          14       1.00      0.97      0.98        95
          15       0.93      0.90      0.92        61
          16       0.97      0.94      0.96        34
          17       1.00    

In [92]:
run.finish()

Acc,0.90973
F1,0.90787
Precision,0.91133
Recall,0.90973
